In [ ]:
from pathlib import Path
import shutil

# Setup Configuration

In [ ]:
SEED = 777

HOME = Path('C:/temp/test')
DATASET_DIR = Path('C:/temp/test/dataset')

# 검출 class 리스트
CLASSES = ['pistol']

# Yolo V5 설정
NAME = 'pistol'
YOLOV5_DIR = HOME / 'yolov5'
BATCH_SIZE = 128
EPOCHS = 50
IMAGE_SIZE = 416
YOLO_V5 = 'yolov5s'
MODEL = YOLOV5_DIR / 'models' / f"{YOLO_V5}.yaml"
WEIGHTS = YOLOV5_DIR / 'weights' / f"{YOLO_V5}.pt"
#WEIGHTS = '\'\''

# training 전에 기존 training 정보가 기록된 디렉토리 삭제 여부
REMOVE_PREV_RUNS = True
# training에 필요한 python library 설치 여부
INSTALL_REQUIREMENTS = False

# 'data.yaml' configuration
DATA_YAML = HOME / 'data.yaml'
n_classes = len(CLASSES)
class_expr = 'names: {}'.format(CLASSES)

# Download YOLO v5 Library

In [ ]:
%cd {HOME}

!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
if INSTALL_REQUIREMENTS:
    %cd {YOLOV5_DIR}
    
    !pip install -r requirements.txt

# Creating `data.yaml` file

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cd {DATA_YAML.parents[0]}

train_images_dir = DATASET_DIR / 'train' / 'images'
valid_images_dir = DATASET_DIR / 'valid' / 'images'

In [ ]:
%%writetemplate data.yaml
train: {train_images_dir}
val: {valid_images_dir}
    
nc: {n_classes}
{class_expr}

# Train a Model
## Remove the previous 'runs'

In [ ]:
if REMOVE_PREV_RUNS:
    runs_path = YOLOV5_DIR / 'runs'
    shutil.rmtree(runs_path, ignore_errors=True)

## Create train command

In [ ]:
%cd {YOLOV5_DIR}

COMMAND = (f'train.py --img {IMAGE_SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} --data {DATA_YAML} '
            f'--cfg {MODEL} --weights {WEIGHTS} --name {NAME} --cache')

print('python ' + COMMAND)

In [ ]:
!python {COMMAND}

In [ ]:
TRAINED_WEIGHT = YOLOV5_DIR / 'runs' / 'train' / NAME / 'weights' / 'best.pt'
DEST_WEIGHT = HOME / f'{NAME}_best.pt'
shutil.copyfile(TRAINED_WEIGHT, DEST_WEIGHT)

In [ ]:
test_dir = DATASET_DIR / 'test' / 'images'
if test_dir.exists():
    COMMAND = (f'detect.py --weights {DEST_WEIGHT} --img {IMAGE_SIZE} --conf 0.1 --line-thickness 2 '
               f'--source {test_dir}')
    print('python ' + COMMAND)

In [ ]:
if test_dir.exists():
    !python {COMMAND}